Lock, Deadlock, Race condition, Thread synchronization <br>
용어 설명<br>
1. Semaphore: 프로세스 간 공유된 자원에 접근하면 문제가 생길 수 있으므로 한 개의 프로세스만 접근할 수 있도록 한다. (경쟁 상태 예방)
2. Mutex: 공유된 자원의 데이터를 여러 thread가 접근하는 것을 막는다. 경쟁 상태를 예방한다.
3. Lock: 상호 배제를 위한 잠금 처리 -> 데이터 경쟁
4. Deadlock: 프로세가 자원을 획득하지 못해 다음 처리를 못하는 무한 대기 상황 (교착상태)
5. Thread synchronization: 안정적으로 동작하게 하기 위해 동기화한다. 동기화 메소드, 동기화 블락
6. Semaphore와 Mutex의 차이
    - Semaphore는 Mutex가 될 수 있지만 그 반대는 안 된다.
    - mutex는 단일 thread가 리소스 또는 중요 섹션을 소비하도록 한다.
    - semaphore는 리소스에 대한 제한된 수의 동시 접근을 허용한다.

In [1]:
import logging
from concurrent.futures import ThreadPoolExecutor
import time

In [2]:
# 문제편

class FakeDataStore:
    
    # 공유변수(value): code, heap, data 영역
    # stack 영역만 각자 갖는다. 즉, 각 스레드는 실행된 함수 영역이 다르다.

    def __init__(self):
        
        self.value = 0

    def update(self, n):
        logging.info(f'Thread {n}: starting update')

        # mutex & lock 등 동기화가 필요한 곳이다.
        local_copy = self.value

        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy

        logging.info(f'Thread {n}: finishing update')

if __name__ == "__main__":
    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    
    # class 인스턴스화
    store = FakeDataStore()

    logging.info(f'Testing update. Starting value is {store.value}')

    # with context 시작
    with ThreadPoolExecutor(max_workers=2) as executor:

        for n in ['First', 'Second', 'Third']:
            executor.submit(store.update, n)


    logging.info(f'Testing update. Ending value is {store.value}')
    # 결과가 3이 되어야 한다.

12:10:44: Testing update. Starting value is 0
12:10:44: Thread First: starting update
12:10:44: Thread Second: starting update
12:10:44: Thread Second: finishing update
12:10:44: Thread First: finishing update
12:10:44: Thread Third: starting update
12:10:44: Thread Third: finishing update
12:10:44: Testing update. Ending value is 2


In [3]:
import threading

# 해결편

class FakeDataStore:
    
    # 공유변수(value): code, heap, data 영역
    # stack 영역만 각자 갖는다. 즉, 각 스레드는 실행된 함수 영역이 다르다.

    def __init__(self):
        
        self.value = 0
        self._lock = threading.Lock()

    def update(self, n):
        logging.info(f'Thread {n}: starting update')

        # mutex & lock 등 동기화가 필요한 곳이다.

        # Lock 획득 (방법1)
        self._lock.acquire()
        logging.info(f'Thread {n} has lock.')
        local_copy = self.value

        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        # lock 반환
        logging.info(f'Thread {n} releases lock.')
        self._lock.release()


        logging.info(f'Thread {n}: finishing update')

if __name__ == "__main__":
    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    
    # class 인스턴스화
    store = FakeDataStore()

    logging.info(f'Testing update. Starting value is {store.value}')

    # with context 시작
    with ThreadPoolExecutor(max_workers=2) as executor:

        for n in ['First', 'Second', 'Third']:
            executor.submit(store.update, n)


    logging.info(f'Testing update. Ending value is {store.value}')
    # 결과가 3이 되어야 한다.

12:10:44: Testing update. Starting value is 0
12:10:44: Thread First: starting update
12:10:44: Thread Second: starting update
12:10:44: Thread First has lock.
12:10:44: Thread First releases lock.
12:10:44: Thread First: finishing update
12:10:44: Thread Second has lock.
12:10:44: Thread Third: starting update
12:10:44: Thread Second releases lock.
12:10:44: Thread Second: finishing update
12:10:44: Thread Third has lock.
12:10:44: Thread Third releases lock.
12:10:44: Thread Third: finishing update
12:10:44: Testing update. Ending value is 3


In [4]:
import threading

# 해결편

class FakeDataStore:
    
    # 공유변수(value): code, heap, data 영역
    # stack 영역만 각자 갖는다. 즉, 각 스레드는 실행된 함수 영역이 다르다.

    def __init__(self):
        
        self.value = 0
        self._lock = threading.Lock()

    def update(self, n):
        logging.info(f'Thread {n}: starting update')

        # mutex & lock 등 동기화가 필요한 곳이다.

        # Lock 획득 (방법2)
        
        with self._lock:
            logging.info(f'Thread {n} has lock.')
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.info(f'Thread {n} releases lock.')


        logging.info(f'Thread {n}: finishing update')

if __name__ == "__main__":
    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    
    # class 인스턴스화
    store = FakeDataStore()

    logging.info(f'Testing update. Starting value is {store.value}')

    # with context 시작
    with ThreadPoolExecutor(max_workers=2) as executor:

        for n in ['First', 'Second', 'Third']:
            executor.submit(store.update, n)


    logging.info(f'Testing update. Ending value is {store.value}')
    # 결과가 3이 되어야 한다.

12:10:45: Testing update. Starting value is 0
12:10:45: Thread First: starting update
12:10:45: Thread Second: starting update
12:10:45: Thread First has lock.
12:10:45: Thread First releases lock.
12:10:45: Thread First: finishing update
12:10:45: Thread Second has lock.
12:10:45: Thread Third: starting update
12:10:45: Thread Second releases lock.
12:10:45: Thread Second: finishing update
12:10:45: Thread Third has lock.
12:10:45: Thread Third releases lock.
12:10:45: Thread Third: finishing update
12:10:45: Testing update. Ending value is 3
